In [1]:
import tensorflow as tf
# #指定使用那块GUP训练
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()
# 设置最大占有GPU不超过显存的70%
config.gpu_options.per_process_gpu_memory_fraction = 0.7 
# # 重点：设置动态分配GPU
config.gpu_options.allow_growth = True
# 创建session时
tf.Session(config=config) 

In [6]:
import numpy as np
from keras.datasets import cifar10
from torch.utils.data import Dataset
import torch.utils.data as Data
class MyDataset(Dataset):
    def __init__(self, imgs, transform=None):
        # super().__init__()
        self.imgs = imgs
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        img = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img)
        img=img.reshape([3,32,32])
        return img
import torch
from torch.autograd import Variable
import model
import torch.nn.functional as F
model = model.cifar10(128)
model.load_state_dict(torch.load('./log/default/best-85.pth'))
model.cuda()
def get_mean_var(y_logit):
    y_logit=np.abs(y_logit-0.1)
    return np.mean(y_logit,axis=1)

def get_possibility(images):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=100, shuffle=True)
    y_logits = []
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]
        y_logits=np.array(y_logits)
    return y_logits
import os
if not os.path.isdir('saved_models_{}'.format('bgan')):
    os.mkdir('saved_models_{}'.format('bgan'))
f = open('saved_models_{}/log_collapse1.txt'.format('bgan'), mode='w')
import torch.utils.data as Data
import cv2

from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import keras.backend as K
from PIL import Image
import matplotlib.pyplot as plt
from keras.utils.generic_utils import Progbar
import sys

import numpy as np
import os
class BGAN():
    """Reference: https://wiseodd.github.io/techblog/2017/03/07/boundary-seeking-gan/"""
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss=self.boundary_loss, optimizer=optimizer)
    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        # model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def boundary_loss(self, y_true, y_pred):
        """
        Boundary seeking loss.
        Reference: https://wiseodd.github.io/techblog/2017/03/07/boundary-seeking-gan/
        """
        return 0.5 * K.mean((K.log(y_pred) - K.log(1 - y_pred))**2)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (X_test, _) = cifar10.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        nb_batches = int(X_train.shape[0] / batch_size)
        global_step=0
        steps = []
        values = []

        for epoch in range(epochs):
            # ---------------------
            #  Train Discriminator
            # ---------------------
            for index in range(nb_batches):
                global_step+=1
                # progress_bar.update(index)

                # get a batch of real images
                image_batch = X_train[index * batch_size:(index + 1) * batch_size]
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Generate a batch of new images
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(image_batch, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d global_step: %d[D loss: %f, acc.: %.2f%%] [G loss: %f]"
                      % (epoch,global_step, d_loss[0], 100 * d_loss[1], g_loss))

                if global_step % sample_interval == 0:
                    self.mode_drop(epoch, global_step)
                    

    def mode_drop(self, epoch, global_step):
        r, c = 10, 1000
        noise = np.random.normal(0, 1, (r * c, 100))
        # sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        y_logits = get_possibility(gen_imgs)
        metrics = get_mean_var(y_logits)


        f.writelines('\n')
        f.writelines('global_step:' + str(global_step))
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in metrics)
        f.writelines('\n')
        
        


if __name__ == '__main__':
    bgan = BGAN()
    bgan.train(epochs=50, batch_size=64, sample_interval=200)


Sequential(
  (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (5): ReLU()
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (9): ReLU()
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (12): ReLU()
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): BatchNorm2d(512, eps=1e-05

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 16, 16, 32)        896       
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 8, 8, 64)          18496     
_________________________________________________________________
batch_normalization_21 (Batc (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 8, 8, 64)          0         
__________

epoch:0 global_step: 55[D loss: 0.542126, acc.: 75.00%] [G loss: 1.788778]
epoch:0 global_step: 56[D loss: 0.455411, acc.: 78.91%] [G loss: 1.010900]
epoch:0 global_step: 57[D loss: 0.378751, acc.: 82.03%] [G loss: 0.483712]
epoch:0 global_step: 58[D loss: 0.307417, acc.: 86.72%] [G loss: 0.727214]
epoch:0 global_step: 59[D loss: 0.214459, acc.: 92.97%] [G loss: 0.776246]
epoch:0 global_step: 60[D loss: 0.297661, acc.: 89.06%] [G loss: 0.902677]
epoch:0 global_step: 61[D loss: 0.226453, acc.: 92.97%] [G loss: 1.335894]
epoch:0 global_step: 62[D loss: 0.204585, acc.: 94.53%] [G loss: 1.063806]
epoch:0 global_step: 63[D loss: 0.253601, acc.: 90.62%] [G loss: 1.189880]
epoch:0 global_step: 64[D loss: 0.358684, acc.: 85.94%] [G loss: 1.053771]
epoch:0 global_step: 65[D loss: 0.385831, acc.: 80.47%] [G loss: 2.029335]
epoch:0 global_step: 66[D loss: 0.384010, acc.: 84.38%] [G loss: 1.633089]
epoch:0 global_step: 67[D loss: 0.311543, acc.: 89.06%] [G loss: 2.093854]
epoch:0 global_step: 68[D

epoch:0 global_step: 166[D loss: 0.453692, acc.: 78.91%] [G loss: 1.978447]
epoch:0 global_step: 167[D loss: 0.469612, acc.: 76.56%] [G loss: 1.531973]
epoch:0 global_step: 168[D loss: 0.450228, acc.: 79.69%] [G loss: 1.984747]
epoch:0 global_step: 169[D loss: 0.420146, acc.: 80.47%] [G loss: 2.075551]
epoch:0 global_step: 170[D loss: 0.596529, acc.: 72.66%] [G loss: 1.408930]
epoch:0 global_step: 171[D loss: 0.428647, acc.: 81.25%] [G loss: 1.581111]
epoch:0 global_step: 172[D loss: 0.452303, acc.: 78.91%] [G loss: 1.169137]
epoch:0 global_step: 173[D loss: 0.380145, acc.: 82.03%] [G loss: 1.492563]
epoch:0 global_step: 174[D loss: 0.446493, acc.: 78.12%] [G loss: 1.232317]
epoch:0 global_step: 175[D loss: 0.356342, acc.: 88.28%] [G loss: 1.623616]
epoch:0 global_step: 176[D loss: 0.318167, acc.: 90.62%] [G loss: 1.522095]
epoch:0 global_step: 177[D loss: 0.373845, acc.: 85.16%] [G loss: 1.993356]
epoch:0 global_step: 178[D loss: 0.430854, acc.: 81.25%] [G loss: 3.267930]
epoch:0 glob

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:41: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


...

epoch:49 global_step: 38530[D loss: 0.644091, acc.: 65.62%] [G loss: 0.815914]
epoch:49 global_step: 38531[D loss: 0.606790, acc.: 65.62%] [G loss: 0.479945]
epoch:49 global_step: 38532[D loss: 0.566909, acc.: 71.88%] [G loss: 1.200154]
epoch:49 global_step: 38533[D loss: 0.550875, acc.: 72.66%] [G loss: 0.647782]
epoch:49 global_step: 38534[D loss: 0.550561, acc.: 71.09%] [G loss: 0.373581]
epoch:49 global_step: 38535[D loss: 0.625305, acc.: 64.06%] [G loss: 0.871346]
epoch:49 global_step: 38536[D loss: 0.700261, acc.: 61.72%] [G loss: 0.381101]
epoch:49 global_step: 38537[D loss: 0.662727, acc.: 57.81%] [G loss: 0.449818]
epoch:49 global_step: 38538[D loss: 0.573975, acc.: 72.66%] [G loss: 0.903371]
epoch:49 global_step: 38539[D loss: 0.575144, acc.: 72.66%] [G loss: 0.950899]
epoch:49 global_step: 38540[D loss: 0.655710, acc.: 57.03%] [G loss: 0.446852]
epoch:49 global_step: 38541[D loss: 0.413629, acc.: 86.72%] [G loss: 0.654304]
epoch:49 global_step: 38542[D loss: 0.618825, acc.: 

epoch:49 global_step: 38639[D loss: 0.609159, acc.: 63.28%] [G loss: 0.478912]
epoch:49 global_step: 38640[D loss: 0.634086, acc.: 65.62%] [G loss: 0.454053]
epoch:49 global_step: 38641[D loss: 0.700125, acc.: 57.81%] [G loss: 0.767635]
epoch:49 global_step: 38642[D loss: 0.752125, acc.: 53.12%] [G loss: 0.412630]
epoch:49 global_step: 38643[D loss: 0.606054, acc.: 64.84%] [G loss: 0.604942]
epoch:49 global_step: 38644[D loss: 0.651096, acc.: 63.28%] [G loss: 0.693934]
epoch:49 global_step: 38645[D loss: 0.494008, acc.: 76.56%] [G loss: 0.896363]
epoch:49 global_step: 38646[D loss: 0.729775, acc.: 55.47%] [G loss: 1.049688]
epoch:49 global_step: 38647[D loss: 0.556320, acc.: 73.44%] [G loss: 0.602911]
epoch:49 global_step: 38648[D loss: 0.644040, acc.: 64.84%] [G loss: 0.576426]
epoch:49 global_step: 38649[D loss: 0.591034, acc.: 68.75%] [G loss: 0.705541]
epoch:49 global_step: 38650[D loss: 0.349165, acc.: 94.53%] [G loss: 0.941708]
epoch:49 global_step: 38651[D loss: 0.842249, acc.: 

epoch:49 global_step: 38747[D loss: 0.526430, acc.: 73.44%] [G loss: 0.689442]
epoch:49 global_step: 38748[D loss: 0.625644, acc.: 62.50%] [G loss: 0.546075]
epoch:49 global_step: 38749[D loss: 0.544190, acc.: 72.66%] [G loss: 0.688377]
epoch:49 global_step: 38750[D loss: 0.747441, acc.: 55.47%] [G loss: 0.488737]
epoch:49 global_step: 38751[D loss: 0.529069, acc.: 71.88%] [G loss: 0.552498]
epoch:49 global_step: 38752[D loss: 0.611339, acc.: 70.31%] [G loss: 0.911542]
epoch:49 global_step: 38753[D loss: 0.560912, acc.: 68.75%] [G loss: 0.725341]
epoch:49 global_step: 38754[D loss: 0.548941, acc.: 75.00%] [G loss: 0.526721]
epoch:49 global_step: 38755[D loss: 0.675012, acc.: 55.47%] [G loss: 0.598977]
epoch:49 global_step: 38756[D loss: 0.643051, acc.: 67.97%] [G loss: 0.292012]
epoch:49 global_step: 38757[D loss: 0.599309, acc.: 68.75%] [G loss: 0.369642]
epoch:49 global_step: 38758[D loss: 0.544556, acc.: 74.22%] [G loss: 0.322654]
epoch:49 global_step: 38759[D loss: 0.527319, acc.: 

epoch:49 global_step: 38855[D loss: 0.632784, acc.: 65.62%] [G loss: 0.832031]
epoch:49 global_step: 38856[D loss: 0.494416, acc.: 81.25%] [G loss: 0.901178]
epoch:49 global_step: 38857[D loss: 0.517042, acc.: 76.56%] [G loss: 0.528985]
epoch:49 global_step: 38858[D loss: 0.534154, acc.: 74.22%] [G loss: 1.170955]
epoch:49 global_step: 38859[D loss: 0.483052, acc.: 76.56%] [G loss: 1.312519]
epoch:49 global_step: 38860[D loss: 0.548197, acc.: 71.88%] [G loss: 1.193776]
epoch:49 global_step: 38861[D loss: 0.645150, acc.: 59.38%] [G loss: 0.686787]
epoch:49 global_step: 38862[D loss: 0.680005, acc.: 61.72%] [G loss: 0.842094]
epoch:49 global_step: 38863[D loss: 0.538049, acc.: 68.75%] [G loss: 0.345281]
epoch:49 global_step: 38864[D loss: 0.397167, acc.: 87.50%] [G loss: 0.809550]
epoch:49 global_step: 38865[D loss: 0.799677, acc.: 51.56%] [G loss: 0.283985]
epoch:49 global_step: 38866[D loss: 0.414963, acc.: 86.72%] [G loss: 0.893854]
epoch:49 global_step: 38867[D loss: 0.630870, acc.: 

epoch:49 global_step: 38961[D loss: 0.778313, acc.: 50.00%] [G loss: 0.666557]
epoch:49 global_step: 38962[D loss: 0.525047, acc.: 75.78%] [G loss: 0.559945]
epoch:49 global_step: 38963[D loss: 0.755775, acc.: 50.00%] [G loss: 0.535618]
epoch:49 global_step: 38964[D loss: 0.498285, acc.: 81.25%] [G loss: 0.838116]
epoch:49 global_step: 38965[D loss: 0.986740, acc.: 41.41%] [G loss: 0.393388]
epoch:49 global_step: 38966[D loss: 0.690693, acc.: 56.25%] [G loss: 0.533358]
epoch:49 global_step: 38967[D loss: 0.607547, acc.: 69.53%] [G loss: 0.491372]
epoch:49 global_step: 38968[D loss: 0.628983, acc.: 65.62%] [G loss: 0.460186]
epoch:49 global_step: 38969[D loss: 0.522991, acc.: 76.56%] [G loss: 0.222219]
epoch:49 global_step: 38970[D loss: 0.857611, acc.: 39.84%] [G loss: 0.697664]
epoch:49 global_step: 38971[D loss: 0.543325, acc.: 72.66%] [G loss: 0.637154]
epoch:49 global_step: 38972[D loss: 0.707524, acc.: 54.69%] [G loss: 0.810850]
epoch:49 global_step: 38973[D loss: 0.493330, acc.: 